#### import necessary packages:

In [1]:
import numpy as np
import sqlite3

### Task 1:
#### Construct function, which computes total buy and sell volumes.

In [209]:
# First take a look at the table.

conn = sqlite3.connect('trades.sqlite')
cur = conn.cursor()

rows = cur.execute("SELECT * FROM epex_12_20_12_13").fetchall()

for row in rows:
    print(row)

('trade_1', 5, 20.0, 'buy', 'strategy_1')
('trade_2', 17, 10.0, 'sell', 'strategy_1')
('trade_3', 15, 8.0, 'buy', 'strategy_1')
('trade_4', 13, 35.0, 'sell', 'strategy_2')
('trade_5', 23, 11.0, 'sell', 'strategy_2')
('trade_6', 8, 20.0, 'buy', 'strategy_2')


In [197]:
# construct functions to calculate buy and sell volume from the table.

# you could potentially also put database name and table name into the arguments of the function to make it more flexible,
# but the way I understand the given task, the functions are supposed to look like this.

def compute_total_buy_volume(*args, **kwargs):
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    buy_quantities = cur.execute("SELECT quantity FROM epex_12_20_12_13 WHERE side = 'buy'").fetchall()


    return(np.sum(buy_quantities))

def compute_total_sell_volume(*args, **kwargs):
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    sell_quantities = cur.execute("SELECT quantity FROM epex_12_20_12_13 WHERE side = 'sell'").fetchall()


    return(np.sum(sell_quantities)) 

In [204]:
print('Total buy volume:', compute_total_buy_volume(), 'MWh')
print('Total sell volume:', compute_total_sell_volume(), 'MWh')

Total buy volume: 28 MWh
Total sell volume: 53 MWh


In [170]:
# #alternative 

# def compute_total_buy_volume(db_name, table_name, *args, **kwargs):
#     conn = sqlite3.connect(db_name)
#     cur = conn.cursor()
#     buy_quantities = cur.execute("SELECT quantity FROM {} WHERE side = 'buy'".format(table_name)).fetchall()


#     return(np.sum(buy_quantities))


# compute_total_buy_volume(db_name='trades.sqlite', table_name='epex_12_20_12_13')

### Task 2:
#### Write a function that computes the PnL of each strategy.

In [181]:
def compute_pnl(strategy_id: str, *args, **kwargs):
    
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    buys = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'buy' AND strategy = '{}'".format(strategy_id)).fetchall()
    sells = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'sell' AND strategy = '{}'".format(strategy_id)).fetchall()
    
    pnl = - np.sum([np.prod(buys[i]) for i in range(len(buys))]) + np.sum([np.prod(sells[i]) for i in range(len(sells))])


    return(pnl)

In [195]:
print('strategy 1 PnL:', compute_pnl(strategy_id='strategy_1'), 'EUR')
print('strategy 2 PnL:', compute_pnl(strategy_id='strategy_2'), 'EUR')

strategy 1 PnL: -50.0 EUR
strategy 2 PnL: 548.0 EUR


The above tool gives the PnL, which might or might not be insightful depending on usecase. It is important to note that The volume of buys and sells for each strategy is not equal. For strategy 1 we buy 20MWh and sell 17MWh and for strategy 2 we buy 8 MWh and sell 36 MWh. Therefore a good/bad PnL does not necessarily imply a good/bad trading strategy.

One interesting indicator to further evaluate strategies could be something like the average buy price and the average sell price of each strategy. It is implemented below.

In [184]:
def compute_avg_buysell(strategy_id: str, *args, **kwargs):
    
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    buys = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'buy' AND strategy = '{}'".format(strategy_id)).fetchall()
    sells = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'sell' AND strategy = '{}'".format(strategy_id)).fetchall()
    
    total_cost_for_buys = np.sum([np.prod(buys[i]) for i in range(len(buys))])
    total_buy_volume = np.sum([buys[i][0] for i in range(len(buys))])

    avg_buy_price = total_cost_for_buys / total_buy_volume

    total_profit_from_sells = np.sum([np.prod(sells[i]) for i in range(len(sells))])
    total_sell_volume = np.sum([sells[i][0] for i in range(len(sells))])

    avg_sell_price = total_profit_from_sells / total_sell_volume

    #return(avg_buy_price, avg_sell_price)
    return {
        'strategy' : strategy_id,
        'avg buy' : avg_buy_price, 
        'avg sell' : avg_sell_price
        }

In [211]:
print(compute_avg_buysell(strategy_id='strategy_1', db_name='trades.sqlite', table_name='epex_12_20_12_13'))
print(compute_avg_buysell(strategy_id='strategy_2', db_name='trades.sqlite', table_name='epex_12_20_12_13'))

{'strategy': 'strategy_1', 'avg buy': 11.0, 'avg sell': 10.0}
{'strategy': 'strategy_2', 'avg buy': 20.0, 'avg sell': 19.666666666666668}


### Task 3:
#### Create web app entry point according to given API definition:

In [ ]:
from flask import Flask, jsonify

In [ ]:
app = Flask(__name__)

@app.route('/v1/pnl/<strategy_id>', methods=['GET'])
def compute_pnl(strategy_id: str):
    
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    
    buys = cur.execute("SELECT quantity, price FROM 'epex_12_20_12_13' WHERE side = 'buy' AND strategy = '{}'".format(strategy_id)).fetchall()
    sells = cur.execute("SELECT quantity, price FROM 'epex_12_20_12_13' WHERE side = 'sell' AND strategy = '{}'".format( strategy_id)).fetchall()
    
    pnl = - np.sum([np.prod(buys[i]) for i in range(len(buys))]) + np.sum([np.prod(sells[i]) for i in range(len(sells))])

    tick = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")

    response_data = {
        "strategy": strategy_id,
        "value": pnl,
        "unit": "euro",
        "capture_time": tick
    }

    return jsonify(response_data), 200

the above code on a test server gives the following responses:

<img src="strat_1.png" width="800">

<img src="strat_2.png" width="800">

<img src="strat_3.png" width="800">